In [1]:
from rdflib import Graph
import os
from urllib.parse import urlparse
import re

In [2]:
graph = "stexpanded"

In [3]:
g = Graph()
g.parse("./_input/rdfxml/" + graph + ".xml", format="xml")

<Graph identifier=Nb03656e0f4cd423080cb88b133e94bfd (<class 'rdflib.graph.Graph'>)>

In [4]:
property_str = None

for prefix, namespace in g.namespaces():
    if str(prefix) == "ns1":
        property_str = str(namespace)

resource_str = property_str.replace("property", "resource")
class_str = property_str.replace("property", "class")

In [5]:
namespace_list = [
    str(namespace)
    for prefix, namespace in g.namespaces()
    if str(namespace) == property_str or not str(namespace).startswith(property_str)
]

In [6]:
def clean_value(value):
    s = str(value)
    s = s.replace(resource_str, "")
    s = s.replace(class_str, "")
    s = re.sub(r'\n+', '', s)
    for sub in namespace_list:
        s = s.replace(sub, " ")
    return s.strip()

def is_image_url(url):
    image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".svg", ".webp", ".tiff"}
    path = urlparse(url).path
    ext = os.path.splitext(path.lower())[1]
    return ext in image_extensions

def is_wikiPageWikiLink(value):
    return "wikiPageWikiLink" in str(value)

In [7]:
test_entity = "http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Magna_Roma"
# test_entity = "http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/William_T._Riker"
# test_entity = "http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/James_T._Kirk_(Phase_II)"

In [8]:
printed_subject = False
abstract_str = ""

for subj, pred, obj in g:
    if str(subj) == test_entity:
        if is_image_url(subj) or is_image_url(obj) or is_wikiPageWikiLink(pred):
            continue
        cleaned_subj = clean_value(subj)
        cleaned_pred = clean_value(pred)
        cleaned_obj  = clean_value(obj)

        if cleaned_pred == "comment":
            continue

        if cleaned_pred == "abstract":
            abstract_str = cleaned_obj
            continue

        if not printed_subject:
            print(f"{cleaned_subj}")
            # print(f"{subj}")
            printed_subject = True

        print(f" - {cleaned_pred}: {cleaned_obj}")

print(f" - abstract: {abstract_str}")

Magna_Roma
 - type: planet_info
 - subject: Category:Class_M_planets
 - planetName: Magna Roma
 - surface: Habitable
 - label: Magna Roma
 - subject: Category:Alpha_Quadrant_planets
 - system: UFC-892
 - altLabel: Magna Roma
 - class: Class_M_planet
 - altLabel: 892-IV
 - abstract: UFC 892-IV, called Magna Roma ("Great Rome") by its inhabitants, is a class M planet where evidence of Hodgkin's Law of Parallel Planetary Development was present in the 23rd century as a parallel of the Roman Empire existed there. (: "Bread and Circuses")In Orion Press continuity, the planet was called "Tellus" and System 892 had the Bayer classification of Beta Hydrae. It had been speculated that the planet's inhabitants had been transplanted from Earth by the Preservers 2,500 years before. (Orion Press: "The Return", TOS novel: Ex Machina)By the late 24th century, the Magna Romans had joined the Federation. (TNG novel: The Captains' Honor)
